In [1]:
!pip install git+https://github.com/huggingface/transformers@82a06db03535c49aa987719ed0746a76093b1ec4

  Cloning https://github.com/huggingface/transformers (to revision 82a06db03535c49aa987719ed0746a76093b1ec4) to /tmp/pip-req-build-t9cur_ga
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-t9cur_ga
  Running command git rev-parse -q --verify 'sha^82a06db03535c49aa987719ed0746a76093b1ec4'
  Running command git fetch -q https://github.com/huggingface/transformers 82a06db03535c49aa987719ed0746a76093b1ec4
  Running command git checkout -q 82a06db03535c49aa987719ed0746a76093b1ec4
  Resolved https://github.com/huggingface/transformers to commit 82a06db03535c49aa987719ed0746a76093b1ec4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.57.1.dev0-py3-none-any.whl size=12023110 sha256=b92e486243673a8b24e3045e9f36c89f311f96e71283764deace9c2ce3d938d0
  Stored in directory: /root/.cache/pip

In [2]:
!pip install fastapi uvicorn python-multipart pyngrok openpyxl pdf2image

In [3]:

# CELL 2: Load HunyuanOCR Model
# ============================================================================
import os
import json
import torch
from PIL import Image
from transformers import AutoProcessor, HunYuanVLForConditionalGeneration

# Model Configuration
MODEL_NAME = "tencent/HunyuanOCR"
MAX_IMAGE_SIDE = 1024
MAX_NEW_TOKENS = 2000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("Loading HunyuanOCR processor and model (FP16)...")
processor = AutoProcessor.from_pretrained(MODEL_NAME, use_fast=False)

model = HunYuanVLForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    attn_implementation="eager",
)

try:
    model = model.to(dtype=torch.float16)
except Exception:
    pass

try:
    model.gradient_checkpointing_enable()
except Exception:
    pass

print("✅ Model & processor loaded successfully!")
print(f"Device: {DEVICE}")
print(f"Device map: {getattr(model, 'hf_device_map', 'unknown')}")

Loading HunyuanOCR processor and model (FP16)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

✅ Model & processor loaded successfully!
Device: cuda
Device map: {'': 0}


In [ ]:
# ============================================================================
# CELL 3: OCR Inference Function
# ============================================================================
def load_and_resize(img, max_size=MAX_IMAGE_SIDE):
    """Resize image if needed"""
    if isinstance(img, str):
        img = Image.open(img).convert("RGB")
    
    w, h = img.size
    scale = max(w, h) / float(max_size)
    if scale > 1.0:
        new_w, new_h = int(w / scale), int(h / scale)
        img = img.resize((new_w, new_h), Image.LANCZOS)
    return img

def clean_repeated_substrings(text):
    """Remove repeated suffix patterns"""
    n = len(text)
    if n < 8000:
        return text
    for length in range(2, n // 10 + 1):
        candidate = text[-length:]
        count = 0
        i = n - length
        while i >= 0 and text[i:i + length] == candidate:
            count += 1
            i -= length
        if count >= 10:
            return text[:n - length * (count - 1)]
    return text

def run_ocr_inference(image_path):
    """Run OCR inference on a single image"""
    try:
        image = load_and_resize(image_path, max_size=MAX_IMAGE_SIDE)
        
        messages = [
            [
                {"role": "system", "content": ""},
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image_path},
                        {"type": "text", "text":
                            (
                               """• Identify the formula in the image and represent it using LaTeX format.

                                  • Parse the table in the image into HTML.

                                  • Parse the chart in the image; use Mermaid format for flowcharts and Markdown for other charts.

                                  • Extract all information from the main body of the document image and represent it in markdown format. Tables should be expressed in HTML format, formulas in the document should be represented using LaTeX format, and the parsing should be organized according to the reading order."""
                            )
                        }
                    ],
                }
            ]
        ]
        
        texts = [
            processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
            for msg in messages
        ]
        
        inputs = processor(
            text=texts,
            images=[image],
            padding=True,
            return_tensors="pt",
        )
        
        model_device = next(model.parameters()).device
        if "pixel_values" in inputs:
            inputs["pixel_values"] = inputs["pixel_values"].to(dtype=torch.float16, device=model_device)
        
        for k, v in list(inputs.items()):
            if k != "pixel_values":
                try:
                    inputs[k] = v.to(device=model_device)
                except Exception:
                    pass
        
        with torch.no_grad():
            if DEVICE == "cuda":
                with torch.autocast(device_type="cuda", dtype=torch.float16):
                    generated_ids = model.generate(
                        **inputs,
                        max_new_tokens=MAX_NEW_TOKENS,
                        do_sample=False,
                        use_cache=True,
                    )
            else:
                generated_ids = model.generate(
                    **inputs,
                    max_new_tokens=MAX_NEW_TOKENS,
                    do_sample=False,
                    use_cache=True,
                )
        
        input_ids = inputs.get("input_ids")
        if input_ids is not None:
            generated_trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(input_ids, generated_ids)]
            decoded = processor.batch_decode(generated_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        else:
            decoded = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        
        output_text = decoded[0] if isinstance(decoded, (list, tuple)) else str(decoded)
        output_text = clean_repeated_substrings(output_text).strip()
        
        # Cleanup
        del inputs, generated_ids
        torch.cuda.empty_cache()
        
        return output_text
        
    except Exception as e:
        print(f"❌ OCR Error: {repr(e)}")
        raise e

print("✅ OCR inference function ready!")

✅ OCR inference function ready!


In [ ]:

# ============================================================================
# CELL 4: Text to Excel Converter (Handles Markdown + HTML Tables)
# ============================================================================
import pandas as pd
import re
from bs4 import BeautifulSoup
from io import BytesIO

def clean_text_line(line):
    """Clean markdown formatting from text lines"""
    line = re.sub(r'\*\*([^*]+)\*\*', r'\1', line)
    line = re.sub(r'\*([^*]+)\*', r'\1', line)
    line = re.sub(r'^#+\s+', '', line)
    return line.strip()

def parse_markdown_table_rows(md_text):
    """Extract ONLY markdown table rows (no text lines)"""
    rows = []
    lines = md_text.split("\n")
    
    for line in lines:
        if re.match(r"^\s*\|.*\|\s*$", line):
            parts = [col.strip() for col in line.strip().split("|")[1:-1]]
            if all(re.match(r"^:?-+:?$", p.replace(" ", "")) for p in parts):
                continue
            rows.append(parts)
    
    return rows

def parse_html_tables(html_text):
    """Extract all HTML tables"""
    soup = BeautifulSoup(html_text, "html.parser")
    all_rows = []
    
    for table in soup.find_all("table"):
        for tr in table.find_all("tr"):
            cells = []
            for td in tr.find_all(["td", "th"]):
                text = td.get_text(strip=True)
                colspan = int(td.get("colspan", 1))
                cells.append(text)
                for _ in range(colspan - 1):
                    cells.append("")
            if cells:
                all_rows.append(cells)
    
    return all_rows

def extracted_text_to_excel(extracted_text):
    """
    Convert extracted text (markdown + HTML tables) to Excel bytes
    Uses proven lab report conversion logic
    """
    from openpyxl.styles import Font, Alignment, Border, Side
    
    rows = []
    
    # Extract table rows separately
    md_table_rows = parse_markdown_table_rows(extracted_text)
    html_table_rows = parse_html_tables(extracted_text)
    
    # Add NON-TABLE TEXT FIRST
    for line in extracted_text.split("\n"):
        # Skip markdown table rows
        if re.match(r"^\s*\|.*\|\s*$", line):
            continue
        
        # Skip HTML table tags
        if any(tag in line for tag in ["<table", "</table", "<tr", "</tr", "<td", "</td", "<th", "</th"]):
            continue
        
        cleaned = clean_text_line(line)
        if cleaned:
            rows.append([cleaned])
        elif line.strip() == "":
            if rows and rows[-1] != [""]:
                rows.append([""])
    
    # Add MARKDOWN TABLE ROWS
    if md_table_rows:
        rows.append([""])
        for table_row in md_table_rows:
            rows.append(table_row)
    
    # Add HTML TABLE ROWS
    if html_table_rows:
        rows.append([""])
        for table_row in html_table_rows:
            rows.append(table_row)
    
    # Clean up
    while rows and rows[-1] == [""]:
        rows.pop()
    
    if not rows:
        rows = [["No content extracted"]]
    
    # Pad rows to same column count
    max_cols = max(len(row) for row in rows)
    padded_rows = [row + [""] * (max_cols - len(row)) for row in rows]
    
    # Create DataFrame
    df = pd.DataFrame(padded_rows).fillna("")
    
    # Convert to Excel
    excel_buffer = BytesIO()
    with pd.ExcelWriter(excel_buffer, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, header=False, sheet_name='OCR_Results')
        
        worksheet = writer.sheets['OCR_Results']
        
        thin_border = Border(
            left=Side(style='thin'),
            right=Side(style='thin'),
            top=Side(style='thin'),
            bottom=Side(style='thin')
        )
        
        # Identify table rows
        table_rows = set()
        for idx, row in enumerate(padded_rows, 1):
            non_empty = sum(1 for cell in row if str(cell).strip())
            if non_empty > 1:
                table_rows.add(idx)
        
        # Auto-adjust column widths
        for column in worksheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
            
            for cell in column:
                try:
                    cell_value = str(cell.value)
                    if cell_value and cell_value not in ["nan", ""]:
                        max_length = max(max_length, len(cell_value))
                except:
                    pass
            
            adjusted_width = min(max(max_length + 2, 12), 50)
            worksheet.column_dimensions[column_letter].width = adjusted_width
        
        # Apply formatting
        for row_idx, row in enumerate(worksheet.iter_rows(), 1):
            is_table_row = row_idx in table_rows
            
            for cell in row:
                cell.alignment = Alignment(wrap_text=True, vertical='top', horizontal='left')
                
                if is_table_row and str(cell.value).strip():
                    cell.border = thin_border
                    
                    if row_idx > 1 and (row_idx - 1) not in table_rows:
                        cell.font = Font(bold=True)
    
    excel_buffer.seek(0)
    return excel_buffer

print("✅ Excel converter function ready!")


✅ Excel converter function ready!


In [6]:
!apt-get update
!apt-get install -y poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]               
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,202 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:9 https://cli.github.com/packages stable/main amd64 Packages [345 B]       
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]     
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease  
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,

In [7]:
# ============================================================================
# CELL 5: PDF to Images Converter
# ============================================================================
from pdf2image import convert_from_path
import tempfile
import shutil

def pdf_to_images(pdf_path):
    """Convert PDF pages to images"""
    try:
        images = convert_from_path(pdf_path, dpi=200)
        image_paths = []
        
        temp_dir = tempfile.mkdtemp()
        
        for i, img in enumerate(images):
            img_path = os.path.join(temp_dir, f"page_{i+1}.jpg")
            img.save(img_path, 'JPEG')
            image_paths.append(img_path)
        
        return image_paths, temp_dir
    except Exception as e:
        print(f"❌ PDF conversion error: {repr(e)}")
        raise e

print("✅ PDF converter function ready!")

✅ PDF converter function ready!


In [11]:
# ============================================================================
# CELL 6: FastAPI Application
# ============================================================================
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import uuid

app = FastAPI(title="HunyuanOCR API")

# Enable CORS for React frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Temp directory for uploads
UPLOAD_DIR = "/content/uploads"
os.makedirs(UPLOAD_DIR, exist_ok=True)

@app.get("/")
def read_root():
    return {"status": "HunyuanOCR API is running!", "model": MODEL_NAME}

@app.post("/upload-and-process")
async def upload_and_process(file: UploadFile = File(...)):
    """
    Upload image/PDF, run OCR, convert to Excel, return Excel file
    """
    try:
        # Validate file type
        file_ext = os.path.splitext(file.filename)[1].lower()
        if file_ext not in [".png", ".jpg", ".jpeg", ".pdf", ".bmp", ".tiff", ".tif"]:
            raise HTTPException(status_code=400, detail="Unsupported file type. Use PNG, JPG, PDF, BMP, or TIFF.")
        
        # Save uploaded file
        file_id = str(uuid.uuid4())
        file_path = os.path.join(UPLOAD_DIR, f"{file_id}{file_ext}")
        
        with open(file_path, "wb") as f:
            content = await file.read()
            f.write(content)
        
        print(f"📁 File saved: {file_path}")
        
        # Process based on file type
        all_extracted_text = []
        temp_dirs_to_cleanup = []
        
        if file_ext == ".pdf":
            # Convert PDF to images
            print("📄 Converting PDF to images...")
            image_paths, temp_dir = pdf_to_images(file_path)
            temp_dirs_to_cleanup.append(temp_dir)
            
            # Run OCR on each page
            for idx, img_path in enumerate(image_paths):
                print(f"🔍 Processing page {idx + 1}/{len(image_paths)}...")
                extracted = run_ocr_inference(img_path)
                all_extracted_text.append(f"--- Page {idx + 1} ---")
                all_extracted_text.append(extracted)
                all_extracted_text.append("")
            
            combined_text = "\n".join(all_extracted_text)
        else:
            # Process single image
            print("🔍 Processing image...")
            combined_text = run_ocr_inference(file_path)
            print("="*60)
            print("EXTRACTED TEXT:")
            print(combined_text)
            print("="*60)
        
        # Convert to Excel
        print("📊 Converting to Excel...")
        excel_buffer = extracted_text_to_excel(combined_text)
        
        # Cleanup
        os.remove(file_path)
        for temp_dir in temp_dirs_to_cleanup:
            shutil.rmtree(temp_dir, ignore_errors=True)
        
        print("✅ Processing complete!")
        
        # Return Excel file
        return StreamingResponse(
            excel_buffer,
            media_type="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
            headers={
                "Content-Disposition": f"attachment; filename={os.path.splitext(file.filename)[0]}_ocr_result.xlsx"
            }
        )
    
    except Exception as e:
        print(f"❌ Error: {repr(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
def health_check():
    return {"status": "healthy", "model_loaded": model is not None}

print("✅ FastAPI app created!")

✅ FastAPI app created!


In [ ]:
# ============================================================================
# CELL 7: Start FastAPI with Ngrok
# ============================================================================
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from threading import Thread

# Allow nested event loops (needed for Colab)
nest_asyncio.apply()

# Set your ngrok auth token (get from https://dashboard.ngrok.com/get-started/your-authtoken)
# Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual token
NGROK_AUTH_TOKEN = "enter your ngrok key"  # ⚠️ REPLACE THIS!

if NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTH_TOKEN":
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("\n" + "="*60)
print("🌐 NGROK PUBLIC URL:")
print(public_url)
print("="*60)
print("\nUse this URL in your React app!")
print("Example: fetch('" + str(public_url) + "/upload-and-process', ...)")
print("\n")

# Run FastAPI server
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

print("✅ FastAPI server is running!")
print("📡 Access the API at:", public_url)
print("📖 API docs at:", str(public_url) + "/docs")
print("\n🔥 Server is ready to accept requests!")

# Keep the cell running
import time
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Server stopped!")
    ngrok.disconnect(public_url)


🌐 NGROK PUBLIC URL:
NgrokTunnel: "https://8581a2fb9278.ngrok-free.app" -> "http://localhost:8000"

Use this URL in your React app!
Example: fetch('NgrokTunnel: "https://8581a2fb9278.ngrok-free.app" -> "http://localhost:8000"/upload-and-process', ...)


✅ FastAPI server is running!
📡 Access the API at: NgrokTunnel: "https://8581a2fb9278.ngrok-free.app" -> "http://localhost:8000"
📖 API docs at: NgrokTunnel: "https://8581a2fb9278.ngrok-free.app" -> "http://localhost:8000"/docs

🔥 Server is ready to accept requests!


INFO:     Started server process [376]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-10' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use

During handling of the

📁 File saved: /content/uploads/b0ed4c93-4e7c-4c10-b137-a6e07228b593.jpg
🔍 Processing image...
EXTRACTED TEXT:
<table><tr><td>Sir</td><td>Ganga Ram</td><td>Hospital</td><td>Dr.</td><td>Raj Kamal</td><td>Agarwal</td></tr><tr><td>Sir</td><td>Ganga Ram</td><td>Hospital</td><td>Rajinder Nagar</td><td>Sr. Consultant</td><td>Sr. Consultant</td></tr><tr><td>New Delhi-110060, INDIA</td><td>Tel : +91-11-4225 4000</td><td></td><td></td><td>Dept. of Anesthesiology</td><td>Dept. of Anesthesiology</td></tr><tr><td>Medical Superintendent (Offig.)</td><td></td><td></td><td></td><td>Pain & Peri Operative Medicine</td><td>Pain & Peri Operative Medicine</td></tr><tr><td>Sir</td><td>Ganga Ram</td><td>Komet Hospital, 7,8, Pura Road</td><td></td><td>Mobile : +91 98101 52201</td><td>Mobile : +91 98101 52201</td></tr><tr><td>New Delhi-110005</td><td>Tel : +91-11-40099998, (D) 45009980</td><td></td><td></td><td colspan="2">E-mail : kamalgarwal300@gmail.com</td></tr></table>

TO WHOM IT MAY CONCERN

As per ICMR

PyngrokNgrokURLError: ngrok client exception, URLError: [Errno 111] Connection refused